### **Resource** 
https://www.datacamp.com/tutorial/open-ai-function-calling-tutorial


## **Function calling trong hàm openai.ChatCompletion.create()**

- GPT là mô hình LLM, xuất ra output dựa trên cảm tính là nhiều. Nên những output sẽ không luôn trật tự và đúng theo cú pháp. 
- Các developer sẽ deal with problem này bằng cách:
    - Add thêm description rõ ràng, expected output vào prompt
    - Add những hàm regex để hậu xử lý kết quả trả về của mô hình

**Ví dụ** 
- câu prompt 1: "David Nguyen is a sophomore majoring in computer science at Stanford University. He is Asian American and has a 3.8 GPA. David is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after graduating."

In [2]:
student_1_description = "David Nguyen is a sophomore majoring in computer science at Stanford University. He is Asian American and has a 3.8 GPA. David is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after graduating."


In [3]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''

In [9]:
import openai
openai.api_key = ""

In [4]:
# Generating response back from gpt-3.5-turbo
openai_response = openai.ChatCompletion.create(
    model = 'gpt-3.5-turbo',
    messages = [{'role': 'user', 'content': prompt1}]
)
response_1 = openai_response['choices'][0]['message']['content']
response_1

'{\n  "name": "David Nguyen",\n  "major": "computer science",\n  "school": "Stanford University",\n  "grades": "3.8 GPA",\n  "club": "Robotics Club"\n}'

In [5]:
import json
json_response = json.loads(response_1)
json_response

{'name': 'David Nguyen',
 'major': 'computer science',
 'school': 'Stanford University',
 'grades': '3.8 GPA',
 'club': 'Robotics Club'}

- câu prompt 2: "Ravi Patel is a sophomore majoring in computer science at the University of Michigan. He is South Asian Indian American and has a 3.7 GPA. Ravi is an active member of the university's Chess Club and the South Asian Student Association. He hopes to pursue a career in software engineering after graduating."

In [4]:
student_2_description="Ravi Patel is a sophomore majoring in computer science at the University of Michigan. He is South Asian Indian American and has a 3.7 GPA. Ravi is an active member of the university's Chess Club and the South Asian Student Association. He hopes to pursue a career in software engineering after graduating."

In [5]:
prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''

In [10]:
import json
openai_response = openai.ChatCompletion.create(
    model = 'gpt-3.5-turbo',
    messages = [{'role': 'user', 'content': prompt2 }]
)

response_2 = openai_response['choices'][0]['message']['content']
json_response = json.loads(response_2)
json_response

{'name': 'Ravi Patel',
 'major': 'computer science',
 'school': 'University of Michigan',
 'grades': '3.7',
 'club': ['Chess Club', 'South Asian Student Association']}

Ta thấy rằng, cùng một description prompt, nhưng kết quả cho ra lại khác nhau.
- Grades: lúc thì có "GPA" lúc thì không
- Club: lúc thì là chuỗi, lúc thì là string. 

**Function Calling** ra đời để thay thế việc ghi description vào thẳng câu prompt ở hai ví dụ trên. Mà ta sẽ định dạng description đó dưới dạng dictionary và function để GPT có thể hiểu rõ hơn. 

Đây là tính năng mà openai add vào library của mình, nên về lý thuyết thì đây là tính năng tối ưu hơn cho việc viết thêm description vào prompt

In [6]:
student_custom_functions = [
    {
        'name': 'get_student_info',
        'description': 'Get the student information from the body of the input text',
        'parameters': {
            'type': 'object',
            'properties': {
                'name': {
                    'type': 'string',
                    'description': 'Name of the person'
                },
                'major': {
                    'type': 'string',
                    'description': 'Major subject.'
                },
                'school': {
                    'type': 'string',
                    'description': 'The university name.'
                },
                'grades': {
                    'type': 'integer',
                    'description': 'GPA of the student.'
                },
                'club': {
                    'type': 'string',
                    'description': 'School club for extracurricular activities. '
                }
                
            }
        }
    }
]

In [18]:
student_description = [student_1_description,student_2_description]
for sample in student_description:
    response = openai.ChatCompletion.create(
        model = 'gpt-3.5-turbo',
        messages = [{'role': 'user', 'content': sample}],
        functions = student_custom_functions,
        function_call = 'auto'
    )

    # Loading the response as a JSON object
    json_response = json.loads(response['choices'][0]['message']['function_call']['arguments'])
    print(json_response)

{'name': 'David Nguyen', 'major': 'computer science', 'school': 'Stanford University', 'grades': 3.8, 'club': 'Robotics Club'}
{'name': 'Ravi Patel', 'major': 'Computer Science', 'school': 'University of Michigan', 'grades': 3.7, 'club': 'Chess Club'}


In [13]:
response['choices'][0]['message']

<OpenAIObject at 0x19ccd4bdf80> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "extract_student_info",
    "arguments": "{\n  \"name\": \"Ravi Patel\",\n  \"major\": \"Computer Science\",\n  \"school\": \"University of Michigan\",\n  \"grades\": 3.7,\n  \"club\": \"Chess Club\",\n  \"club2\": \"South Asian Student Association\"\n}"
  }
}

### **Cách dùng thực sự của Function Calling**

Vì output của OpenAI sau khi qua Function Calling đã được xử lý, nên output có thể được đem vào để chạy ở những tác vụ khác.

Ta sẽ code thêm xíu nữa để có thể làm thêm nhiều tác vụ khác. 

Ví dụ: Sau khi extract xong thông tin, ta muốn in ra thông tin của học sinh bằng hàm *get_student_info()*

In [7]:
def get_student_info(name, major, school, grades, club):
    return f"Student {name} learns at {major} major at {school}. He/she has a grade {grades} GPA and has attended in {club}"

In [10]:
import json
student_description = [student_1_description,student_2_description]
for sample in student_description:
    response = openai.ChatCompletion.create(
        model = 'gpt-3.5-turbo',
        messages = [{'role': 'user', 'content': sample}],
        functions = student_custom_functions,
        function_call = 'auto'
    )
    response_message = response["choices"][0]["message"]
    function_name = response_message['function_call']['name']
    # Tạo ra một dictionary để map string (function name) với object function (get_student_info)
    available_functions = {
            "get_student_info": get_student_info,
    }

    function_args  = json.loads(response_message['function_call']['arguments'])
    function_to_call = available_functions[function_name]
    response_message = function_to_call(*list(function_args.values()))
    print(response_message)

Student David Nguyen learns at Computer Science major at Stanford University. He/she has a grade 3.8 GPA and has attended in Robotics Club
